# Preliminaries

## Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Libraries

In [2]:
# hide cell output
%%capture

# imports
!pip install geopandas
import pandas as pd
import geopandas as gpd
import fiona
import folium
from folium.plugins import TimestampedGeoJson
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
#import simplekml

## Import & process overflight and facility data (raw, takes a long time)

In [33]:
# # overflight data
# this code is really slow. Is there any reason I have to use gpd.read_file instead of a pandas function?
sat = gpd.read_file("/content/drive/MyDrive/CORONA/declassified2.zip")
sat1 = gpd.read_file("/content/drive/MyDrive/CORONA/declassified1.zip")

# # facilities data
fac = pd.read_excel("/content/drive/MyDrive/CORONA/NL_geocoded_working.xlsx")

Transform the facilities dataframe into GeoDataFrame

In [35]:
# Clean up facilities df
fac = fac.drop(fac.columns[3], axis=1) # drop weird NaN column
fac = fac.dropna() # drop rows with missing coordinates, of which there are 11
fac = fac.reset_index(drop=True) # reset now missing indices

# Split Coordinates into two columns
fac['Coordinates'] = fac['Coordinates'].str.replace(',', '.') # deal with coordinate notation inconsistency
split_columns = fac['Coordinates'].str.split('.', expand=True) # split coords by periods
# split_columns[1] = split_columns[1].str[1:] # code was dropping the first digit after decimal point of Latitude. Why?
fac['Latitude'] = split_columns[0] + '.' + split_columns[1]
fac['Longitude'] = split_columns[2] + '.' + split_columns[3]

# Convert Lat and Long to spatial points - note that we are rounding to the nearest integer
# fac['Latitude'] = round(fac['Latitude'].astype(float))
# fac['Longitude'] = round(fac['Longitude'].astype(float))

# Create GeoDataFrame with facility locations
fac_geodf = gpd.GeoDataFrame(
    fac, geometry = gpd.points_from_xy(fac["Latitude"], fac["Longitude"]))

# Set the CRS to avoid CRS conflict with the overflight data
fac_geodf = fac_geodf.set_crs('epsg:4326')

Obtain capture data: Identify instances where a facility's coordinates were photographed in an overflight

In [36]:
# combine the sat dataframes
sat = pd.concat([sat, sat1], ignore_index=True)

In [37]:
# join the two GeoDataFrames
captures = gpd.tools.sjoin(fac_geodf, sat, predicate="within", how='left')

# the resulting GDF contains:
# a) each instance where an overflight photographed a polygon containing the coordinates of an enrichment facility AND
# b) a placeholder entry for each facility which was never photographed (no overlap with fac_geodf). Such entries have NaN or NaT values for all variables from fac_geodf (index_right through SW Corne_3)

# reset the indices and hang on to the facility IDs
captures = captures.reset_index(names = "fac_index")

Save data obtained to this point

In [32]:
# sat.to_csv("/content/drive/MyDrive/CORONA/sat.csv")
# captures.to_csv("/content/drive/MyDrive/CORONA/captures.csv")
# fac_geodf.to_csv("/content/drive/MyDrive/CORONA/fac_geodf.csv")

## Import overflight, facility, & capture data (preprocessed)

In [25]:
# sat = pd.read_csv("/content/drive/MyDrive/CORONA/sat.csv")
# captures = pd.read_csv("/content/drive/MyDrive/CORONA/captures.csv")
# fac_geodf = gpd.read_file("/content/drive/MyDrive/CORONA/fac_geodf.csv",
                          # GEOM_POSSIBLE_NAMES="geometry",
                          # KEEP_GEOM_COLUMNS="NO")

## Filter capture data

Do something about facilities with unknown start dates, denoted with a value of -99 in the dataset

In [40]:
# ---- OPTION 1 ----
# divide into two datasets - REMOVES facilities with unknown start date from analysis
# caps_unknownstart = captures[captures["start_date"] == -99] # capture occurences of facilities with unknown start dates
# captures = captures[(captures["start_date"] != -99) | (captures["start_date"].isna())] # capture occurences of facilities with known start dates AND facilities with no captures

# ---- OPTION 2 ----
# convert unknown start date (-99) to very early start date (1900) - ASSUMES facilities with unkown start date were built early
captures['start_date'] = captures['start_date'].replace(-99,1900)

In [39]:
captures

,fac_index,original_country_name,original_ccode,original_facility_name,Coordinates,start_date,Latitude,Longitude,geometry,index_right,...,NW Corne_2,NW Corne_3,NE Corne_2,NE Corne_3,SE Corne_2,SE Corne_3,SW Corne_2,SW Corne_3,Direction,NW Cormer
0,0,Argentina,160,Ezeiza – SF Reprocessing Facility,-34.854123.-58.522925,1968,-34.854123,-58.522925,POINT (-34.85412 -58.52293),74884.0,...,-39.772,NaN,-51.055,-30.475,-57.973,-26.221,-59.709,-37.186,012M,-52.51
1,0,Argentina,160,Ezeiza – SF Reprocessing Facility,-34.854123.-58.522925,1968,-34.854123,-58.522925,POINT (-34.85412 -58.52293),74885.0,...,-39.568,NaN,-52.552,-29.836,-59.521,-25.194,-61.341,-36.799,012M,-54.057
2,0,Argentina,160,Ezeiza – SF Reprocessing Facility,-34.854123.-58.522925,1968,-34.854123,-58.522925,POINT (-34.85412 -58.52293),74886.0,...,-39.28,NaN,-54.029,-29.082,-61.041,-23.977,-62.965,-36.276,012M,-55.598
3,0,Argentina,160,Ezeiza – SF Reprocessing Facility,-34.854123.-58.522925,1968,-34.854123,-58.522925,POINT (-34.85412 -58.52293),74887.0,...,-39.03,NaN,-55.506,-28.32,-62.558,-22.7,-64.587,-35.775,012M,-57.137
4,0,Argentina,160,Ezeiza – SF Reprocessing Facility,-34.854123.-58.522925,1968,-34.854123,-58.522925,POINT (-34.85412 -58.52293),74888.0,...,-38.7,NaN,-56.965,-27.439,-64.045,-21.21,-66.201,-35.131,012M,-58.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8270,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949,46.0428588,14.4875933,POINT (46.04286 14.48759),627759.0,...,45.455,NaN,14.656,47.548,14.488,47.602,14.275,45.439,138D,14.449
8271,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949,46.0428588,14.4875933,POINT (46.04286 14.48759),772952.0,...,43.95,NaN,14.616,46.35,14.466,46.4,14.1,43.95,104D,14.266
8272,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949,46.0428588,14.4875933,POINT (46.04286 14.48759),869728.0,...,45.745,NaN,14.867,48.02,14.679,48.011,14.341,45.806,251D,14.512
8273,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949,46.0428588,14.4875933,POINT (46.04286 14.48759),869902.0,...,45.739,NaN,14.958,47.951,14.785,48.014,14.435,45.733,251D,14.621


Filter out both capture events which happened before construction started on the facility and placeholders for facilities which were never captured

In [41]:
# convert to datetimes
captures['start_date'] = pd.to_datetime([f"{year}-01-01" for year in captures["start_date"]]).tz_localize(None) # assume construction started on the first day of the year
captures['Acquisitio'] = pd.to_datetime(captures['Acquisitio'])

# add unique indices (0 to n_captures) to each capture occurence, retaining facility indices (0 to n_facilities) as a separate column
# captures = captures.reset_index(names = "fac_index") # this is necessary when not loading captures from saved file
# captures.drop(columns=['Unnamed: 0']) # necessary when loading captures from saved file

# filter the dataset to remove cases:
# a) where the photo was taken before construction started, OR
# b) where the facility's coordinates were never captured at all
captures_filtered = captures[captures['start_date'] <= captures['Acquisitio']]
captures_filtered = captures_filtered.reset_index(drop = True) # reset indices

In [42]:
captures_filtered

,fac_index,original_country_name,original_ccode,original_facility_name,Coordinates,start_date,Latitude,Longitude,geometry,index_right,...,NW Corne_2,NW Corne_3,NE Corne_2,NE Corne_3,SE Corne_2,SE Corne_3,SW Corne_2,SW Corne_3,Direction,NW Cormer
0,5,Belgium,211,"Mol, Purex Reprocessing Facility, Eurochemic",51.194551.5.119991,1960-01-01,51.194551,5.119991,POINT (51.19455 5.11999),74515.0,...,50.563,NaN,8.359,54.57,3.417,55.518,2.639,51.543,008M,7.574
1,5,Belgium,211,"Mol, Purex Reprocessing Facility, Eurochemic",51.194551.5.119991,1960-01-01,51.194551,5.119991,POINT (51.19455 5.11999),74516.0,...,50.682,NaN,6.7,54.689,1.74,55.643,0.958,51.657,008M,5.914
2,5,Belgium,211,"Mol, Purex Reprocessing Facility, Eurochemic",51.194551.5.119991,1960-01-01,51.194551,5.119991,POINT (51.19455 5.11999),177465.0,...,48.072,NaN,9.701,52.874,3.806,53.209,3.513,48.463,024M,9.405
3,5,Belgium,211,"Mol, Purex Reprocessing Facility, Eurochemic",51.194551.5.119991,1960-01-01,51.194551,5.119991,POINT (51.19455 5.11999),177466.0,...,47.968,NaN,7.872,52.75,1.973,53.093,1.679,48.352,024M,7.575
4,5,Belgium,211,"Mol, Purex Reprocessing Facility, Eurochemic",51.194551.5.119991,1960-01-01,51.194551,5.119991,POINT (51.19455 5.11999),177467.0,...,47.862,NaN,6.043,52.629,0.141,52.98,-0.155,48.238,024M,5.746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949-01-01,46.0428588,14.4875933,POINT (46.04286 14.48759),627759.0,...,45.455,NaN,14.656,47.548,14.488,47.602,14.275,45.439,138D,14.449
5948,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949-01-01,46.0428588,14.4875933,POINT (46.04286 14.48759),772952.0,...,43.95,NaN,14.616,46.35,14.466,46.4,14.1,43.95,104D,14.266
5949,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949-01-01,46.0428588,14.4875933,POINT (46.04286 14.48759),869728.0,...,45.745,NaN,14.867,48.02,14.679,48.011,14.341,45.806,251D,14.512
5950,156,Yugoslavia,345,Jozef Stefan Institute near Ljubljana,46.0428588.14.4875933,1949-01-01,46.0428588,14.4875933,POINT (46.04286 14.48759),869902.0,...,45.739,NaN,14.958,47.951,14.785,48.014,14.435,45.733,251D,14.621


# Analysis

How many of the facilities were ever photographed?

In [34]:
# total number of facilities
print("Total number of facilities:",len(fac['original_facility_name'].unique()))

# number of facilities with known start dates
print("Number of facilities with known start dates:",len(captures['original_facility_name'].unique()))

# number of facilities with known start dates which were photographed
print("Number of facilities with known start dates which were photographed:",len(captures_filtered['original_facility_name'].unique()))

Total number of facilities: 169


This means at least 157 - 85 = 72 facilities avoided detection by these CORONA flights. Worth mentioning that a fair number of these (20) were built after the end of the CORONA program.

Let's make some tables & charts summarizing the coverage over time.

In [ ]:
coverage = pd.DataFrame(columns=['year'])

for year in list(range(1959, 1981)):
  year_date = datetime(year, 1, 1)

  # extant facilities
  fac_exist = captures[captures['start_date'] <= year_date] # facilities which were built in or before year
  n_fac_exist = len(fac_exist['original_facility_name'].unique())

  # spotted facilities
  fac_spotted = captures_filtered[captures_filtered['Acquisitio'] <= year_date] # facilities which were spotted on or before Jan 1, year
  n_fac_spotted = len(fac_spotted['original_facility_name'].unique())

  coverage = pd.concat([coverage, pd.DataFrame({"year": [year],
                                                "number of extant facilities": n_fac_exist,
                                                "number of spotted facilities": n_fac_spotted,
                                                "coverage": n_fac_spotted/n_fac_exist})], ignore_index=True)

print(coverage)

TypeError: ignored

In [ ]:
coverage.plot.area(x='year',
                   y=["number of extant facilities", "number of spotted facilities"],
                   stacked = False,
                   color=['#7eb0d5','#fd7f6f'],
                   alpha=0.75)

In [ ]:
captures.to_csv('captures.csv')
captures_filtered.to_csv('captures_filtered.csv')
coverage.to_csv('coverage.csv')

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# pd.reset_option('^display.', silent=True)




# Prepare a map

In [ ]:
#Provide the color column, not entirely necessary
# fac_geodf['fillColor'] = '#53c688'

# assume unknown start dates were built in 1900
fac_geodf['start_date'] = fac_geodf['start_date'].astype(int)
fac_geodf['start_date'] = fac_geodf['start_date'].replace(-99, 1900)

# captures_filtered['fillcolor'] = '#ac3977'

# get end date in milliseconds after Jan 1, 1960
end_date = captures_filtered["Acquisitio"].max().isoformat()
# end_date = (end_date - datetime(1970, 1, 1)).total_seconds()
# end_date = int(end_date * 1000)

,field_1,original_country_name,original_ccode,original_facility_name,Coordinates,start_date,Latitude,Longitude,geometry
0,0,Argentina,160,Ezeiza – SF Reprocessing Facility,-34.854123.-58.522925,1968,-35.0,-59.0,POINT (-35.000 -59.000)
1,1,Argentina,160,Ezeiza II – SF Reprocessing Facility,-34.854123.-58.522925,1978,-35.0,-59.0,POINT (-35.000 -59.000)
2,2,Argentina,160,Pilcaniyeu Enrichment Facility I,-41.1226452.-70.7215709,1979,-41.0,-71.0,POINT (-41.000 -71.000)
3,3,Australia,900,Lucas Heights,-34.0506403.150.9872444,1965,-34.0,151.0,POINT (-34.000 151.000)
4,4,Australia,900,Silex Laser Enrichment Facility at Lucas Heigh...,38.63805199999999.-90.221913,1982,39.0,-90.0,POINT (39.000 -90.000)
...,...,...,...,...,...,...,...,...,...
152,152,United Kingdom,200,MOX Demonstration Facility,54.425604.-3.500048,1900,54.0,-4.0,POINT (54.000 -4.000)
153,153,Yugoslavia,345,Vinca Reprocessing Center near Belgrade (Boris...,44.758497974697725. 20.598281709783674,1956,45.0,21.0,POINT (45.000 21.000)
154,154,Yugoslavia,345,Vinca Electromagnetic Isotope Separator (Vinca...,44.758497974697725. 20.598281709783675,1956,45.0,21.0,POINT (45.000 21.000)
155,155,Yugoslavia,345,Rudjer Boskovic Institute,45.8300113.15.9870591,1952,46.0,16.0,POINT (46.000 16.000)


Map creation functions

In [ ]:
fac_geodf_test = fac_geodf[fac_geodf['start_date'] >= 1970]

,field_1,original_country_name,original_ccode,original_facility_name,Coordinates,start_date,Latitude,Longitude,geometry
1,1,Argentina,160,Ezeiza II – SF Reprocessing Facility,-34.854123.-58.522925,1978,-35.0,-59.0,POINT (-35.00000 -59.00000)
2,2,Argentina,160,Pilcaniyeu Enrichment Facility I,-41.1226452.-70.7215709,1979,-41.0,-71.0,POINT (-41.00000 -71.00000)
4,4,Australia,900,Silex Laser Enrichment Facility at Lucas Heigh...,38.63805199999999.-90.221913,1982,39.0,-90.0,POINT (39.00000 -90.00000)
6,6,Brazil,140,Aerospace Technical Center (Institute of Advan...,-23.248269919363636. -45.85989846071746,1975,-23.0,-46.0,POINT (-23.00000 -46.00000)
7,7,Brazil,140,BRN Enrichment (Aramar Isotopic Enrichment Lab...,-23.393281.-47.6047147,1976,-23.0,-48.0,POINT (-23.00000 -48.00000)
8,8,Brazil,140,"INB Resende –Pilot Enrichment Facility, Rio De...",-22.4722561.-44.4649211,1982,-22.0,-44.0,POINT (-22.00000 -44.00000)
10,10,Brazil,140,Pilot Enrichment Plant- Belo Horizonte (INB Re...,-19.919052.-43.9386685,1975,-20.0,-44.0,POINT (-20.00000 -44.00000)
25,25,Egypt,651,Hot Cell Facility at Inshas Nuclear Research C...,30.387053.31.4440355,1980,30.0,31.0,POINT (30.00000 31.00000)
28,28,France,220,Areva NC La Hague – UP2 – 800,49.6639603.-1.8485699,1982,50.0,-2.0,POINT (50.00000 -2.00000)
29,29,France,220,Areva NC La Hague – UP3,49.6639603.-1.8485699,1982,50.0,-2.0,POINT (50.00000 -2.00000)


In [ ]:
def create_geojson_features():
    features = []

    for _, row in fac_geodf_test.iterrows():
        x = int(row["start_date"])
        # Create a datetime object with the year, and set the month and day to 1
        dt = datetime(x, 1, 1)
        # Get the timestamp in seconds
        # timestamp_seconds = (dt - datetime(1970, 1, 1)).total_seconds()
        timestamp = dt.isoformat()
        # Convert timestamp to milliseconds
        # timestamp_milliseconds = int(timestamp_seconds * 1000)

        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point',
                'coordinates':[float(row['Longitude']),float(row['Latitude'])]
            },
            'properties': {
                'times': [timestamp],  # Display until the end of the animation
                # 'times':[0],
                'style': {'color' : ''},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': 'blue',
                    'fillOpacity': 0.5,
                    'stroke': 'true',
                    'radius': 3
                },
                'name': 'Facilities'
            }
        }
        features.append(feature)

    # for _, row in captures_filtered.iterrows():
    #     y = row["Acquisitio"]
    #     # Get the timestamp in seconds
    #     # timestamp_seconds = (y - datetime(1970, 1, 1)).total_seconds()
    #     timestamp = y.isoformat()
    #     # Convert timestamp to milliseconds
    #     # timestamp_milliseconds = int(timestamp_seconds * 1000)

    #     feature = {
    #         'type': 'Feature',
    #         'geometry': {
    #             'type':'Point',
    #             'coordinates':[float(row['Longitude']),float(row['Latitude'])]
    #             # 'coordinates':[float(row['Latitude']),float(row['Longitude'])]
    #         },
    #         'properties': {
    #             # 'times': [timestamp_milliseconds, timestamp_milliseconds + (30 * 24 * 60 * 60 * 1000)],  # Display for 30 days
    #             # 'times': [timestamp], # display until the end of the animation
    #             'times':[0],
    #             'style': {'color' : ''},
    #             'icon': 'circle',
    #             'iconstyle':{
    #                 'fillColor': '#FF0000',
    #                 'fillOpacity': 0.8,
    #                 'stroke': 'true',
    #                 'radius': 4
    #             },
    #             'name': 'Overflights'
    #         }
    #     }
        # features.append(feature)
    return features

In [ ]:
fac_geojson = create_geojson_features()

# Create the folium map
m = folium.Map()

#Add the slider
# Add TimestampedGeoJson layer to the map
# TimestampedGeoJson({
#     'type': 'FeatureCollection',
#     'features': fac_geojson
# }, period='P1M', duration='P1M', auto_play=False).add_to(m)

fac_geojson = sorted(fac_geojson, key=lambda x: x['properties']['times'][0])

TimestampedGeoJson(fac_geojson,transition_time=200,period='P1M').add_to(m)

# Add legend to the map
folium.LayerControl().add_to(m)

m
# m.save('map_test.html')

In [ ]:
len(fac_geojson)

50

In [ ]:
datetime.fromisoformat(fac_geojson[0]['properties']['times'][0])
# and datetime(x['properties']['times'][0]) < datetime(1973,1,1)
filtered = filter(lambda x: datetime.fromisoformat(x['properties']['times'][0]) > datetime(1965,1,1), fac_geojson)
filtered = list(filtered)

pd.DataFrame.from_records(filtered)

[{'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [2.0, 49.0]},
  'properties': {'times': ['1966-01-01T00:00:00'],
   'style': {'color': ''},
   'icon': 'circle',
   'iconstyle': {'fillColor': 'blue',
    'fillOpacity': 0.5,
    'stroke': 'true',
    'radius': 3},
   'name': 'Facilities'}},
 {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [8.0, 49.0]},
  'properties': {'times': ['1967-01-01T00:00:00'],
   'style': {'color': ''},
   'icon': 'circle',
   'iconstyle': {'fillColor': 'blue',
    'fillOpacity': 0.5,
    'stroke': 'true',
    'radius': 3},
   'name': 'Facilities'}},
 {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [51.0, 36.0]},
  'properties': {'times': ['1967-01-01T00:00:00'],
   'style': {'color': ''},
   'icon': 'circle',
   'iconstyle': {'fillColor': 'blue',
    'fillOpacity': 0.5,
    'stroke': 'true',
    'radius': 3},
   'name': 'Facilities'}},
 {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [

# Scratch

In [ ]:
y = min(captures_filtered["Acquisitio"])

min(fac_geodf["start_date"])

(y - datetime(1970, 1, 1)).total_seconds()

1900

In [ ]:
timestamp_seconds = (y - datetime(1960, 1, 1)).total_seconds()
# Convert timestamp to milliseconds
timestamp_milliseconds = int(timestamp_seconds * 1000)

252460800000
252460800000

252460800000 (1968)
568080000000 (1978)
599616000000 (1979)

# n = number of milliseconds after 1960
n = 599616000000
(n/1000)/(60*60*24*365)

y = fac_geodf['start_date'][0]
dt.isoformat() = datetime(y, 1, 1)
(dt - datetime(1960, 1, 1)).total_seconds()
end_date

'1980-10-02T00:00:00'

Run the map

In [ ]:
fac_geojson = create_geojson_features()
fac_geojson[0]

# Create the folium map
m = folium.Map(
    # location=[56.096555, -3.64746],
    # location=[0,0],
    # tiles="cartodbpositron",
    # zoom_start=2,
    # crs = 'EPSG4326'
)

#Add the slider
# Add TimestampedGeoJson layer to the map
TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': fac_geojson
}, period='P1M', duration='P1M', auto_play=False).add_to(m)

# Add legend to the map
folium.LayerControl().add_to(m)

#save
# m.save('C:/Users/hasko/OneDrive/Plocha/ETH Year 1 Materials/CSS - Nuclear/Map Project/map.html')

m
# m.save('map_test.html')

NameError: ignored

In [ ]:
m = folium.Map(
    location=[56.096555, -3.64746],
    tiles="cartodbpositron",
    zoom_start=2,
)

toy_feat = [{'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-35.0, -59.0]},
  'properties': {'times': [0,0],
   'style': {'color': ''},
   'icon': 'circle',
   'iconstyle': {'fillColor': 'blue',
    'fillOpacity': 0.8,
    'stroke': 'true',
    'radius': 5},
   'name': 'Facilities'}}]

TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': toy_feat
}, period='P1M', duration='P1M', auto_play=False).add_to(m)

# Add legend to the map
folium.LayerControl().add_to(m)
m